 Decision Tree CART Algorithm

-  Classification of the IBM Watson Analytics - Employee Attritionn HR Sample DataSet 


In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import mixture
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

%matplotlib inline

# Pre Process- Data 
# Convert CSV file to TSV Shell Command ==> $  < FileName.csv tr "," "\t" > FileName.tsv


df=pd.read_csv('hr2.tsv',sep='\t')
df.head(5)

# http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
# Read TSV with \t 




,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,3,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,4,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,5,...,4,80,1,6,3,3,2,2,2,2


In [3]:
mymap = {'Yes':1,'No':0,'Travel_Rarely':1, 'Travel_Frequently': 2 ,'Non-Travel':3, 'Research & Development' :1 , 
         'Human Resources':2,'Sales':3,'Life Sciences':1,'Medical':6,'Technical Degree':3,'Marketing':4,'Other':5,
        'Female':1, 'Male':2,'Research Scientist':1,'Laboratory Technician':2,'Healthcare Representative':3,
         'Manufacturing Director':4,'Manager':5,'Sales Representative':6,'Research Director':7,'Sales Executive':8,
        'Single':1,'Married':2,'Divorced':3}#Medical = 6 as HR =2 in another column

#
dfh =df.applymap(lambda s: mymap.get(s) if s in mymap else s)
#

# TBD- LabelEncoder - http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
# In mymap == Yes =1 and No =0 - replacements made in both Attrition and OverTime
# http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.applymap.html
# http://pandas.pydata.org/pandas-docs/stable/indexing.html

#dfh.to_csv('dfh_20DEC.csv') # Ok for down csv 

dfh.head(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,1,1102,3,1,2,1,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,0,2,279,1,8,1,1,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1,1,1373,1,2,2,5,1,3,...,2,80,0,7,3,3,0,0,0,0
3,33,0,2,1392,1,3,4,1,1,4,...,3,80,0,8,3,3,8,7,3,0
4,27,0,1,591,1,2,1,6,1,5,...,4,80,1,6,3,3,2,2,2,2


In [4]:
#InterimDF Dropped-Attr,EmployeeCount,EmployeeNumber ,Over18 and StandardHours
df1 = dfh.drop(df.columns[[1,8,9,21,26]],axis=1,inplace=False) 
#print df1.shape
names = df1.columns.values
print "_"*90
#print names
#print "_"*90
df2 = pd.DataFrame(dfh["Attrition"]) #  Interim DF only - Attr
#names1 = df2.columns.values
#print names1
#print df2.shape 
print df2["Attrition"].value_counts() # Here - 0 == Live Employee , 1 == Exited Employee / Attrited Employee 
#
print "_"*90
frames = [df1, df2]
df3 = pd.concat(frames,axis=1)
print df3.shape 
#print df3.head(5) 
df3.to_csv('dfh3_20DEC.csv') # Ok for down csv 

#df4 = dfh[["MonthlyRate","DailyRate"]] #  Interim DF only for PLotting - MonthlyRate + DailyRate


__________________________________________________________________________________________
0    2466
1     474
Name: Attrition, dtype: int64
__________________________________________________________________________________________
(2940, 31)


In [5]:
# Convert DF to Numpy Array 
# 1st Numpy Array == X , only features 
# 2nd Numpy Array == y , only target Labels
import numpy as np

X = df3.iloc[:,0:31].values # All Features of df3 - TARGET Feature in last Column 
#y = df2.iloc[:,0].values # Not separating a Target Feature for the Decision Tree 
#
print X.shape
#print y.shape
#print('Target Variable "Attrition":', (y))
print "_"*90+"_|||_______"
#print('Class labels for Target Variable "Attrition":', np.unique(X[:,31])) # TBD ?? 
print('Probablity of Class Label ==1 = {:.4f}'.format(df3["Attrition"].mean()))
print('Probablity of Class Label ==0 = {:.4f}'.format(1-df3["Attrition"].mean()))
print "_"*90+"_|||_______"
print "Model that Predicts 83.88% Accuracy is Non Predictor OR NO_Model- as it will always predict Dominant Class"
print "This dataset Dominant Class = ZERO or LIVE EMPLOYEE - we need more than 83.88% Accuracy Score."
print "_"*90+"_|||_______"



(2940, 31)
___________________________________________________________________________________________|||_______
Probablity of Class Label ==1 = 0.1612
Probablity of Class Label ==0 = 0.8388
___________________________________________________________________________________________|||_______
Model that Predicts 83.88% Accuracy is Non Predictor OR NO_Model- as it will always predict Dominant Class
This dataset Dominant Class = ZERO or LIVE EMPLOYEE - we need more than 83.88% Accuracy Score.
___________________________________________________________________________________________|||_______


In [6]:
my_data = X
print my_data

[[  41    1 1102 ...,    0    5    1]
 [  49    2  279 ...,    1    7    0]
 [  37    1 1373 ...,    0    0    1]
 ..., 
 [  27    1  155 ...,    0    3    0]
 [  49    2 1023 ...,    0    8    0]
 [  34    1  628 ...,    1    2    0]]


### __init__ methods can take any number of arguments, and just like functions, the arguments can be defined with # default values.

### Refer URL -- 
https://docs.python.org/2/tutorial/classes.html
    
class decisionnode: # CLASS 
  def __init__(self,col=-1,value=None,results=None,tb=None,fb=None): #_init_ METHOD # col,value,results =ARGUMENTS
    self.col=col
    self.value=value
    self.results=results
    self.tb=tb    # tb= True Branch 
    self.fb=fb    # fb = False Branch


In [7]:
class decisionnode:
  def __init__(self,col=-1,value=None,results=None,tb=None,fb=None): # __init__ methods can take any number of arguments, and just like functions, the arguments can be defined with default values, making them optional to the caller.
    self.col=col
    self.value=value
    self.results=results
    self.tb=tb    # tb= True Branch / Right Branch 
    self.fb=fb    # fb = False Branch / Left B 

# Divides a set on a specific column. Can handle numeric or nominal values
def divideset(rows,column,value):
    # Make a function that tells us if a row is in 
    # the first group (true) or the second group (false)
   split_function=None
   if isinstance(value,int) or isinstance(value,float):   ## IF - OR - ELSE - Loop ........... :P
      split_function=lambda row:row[column]>=value
   else:
      split_function=lambda row:row[column]==value
   
   # Divide the rows into two sets and return them
   set1=[row for row in rows if split_function(row)]
   set2=[row for row in rows if not split_function(row)]
   #print set1               # Dhankar Code 
   #print type(set1)         # Dhankar Code 
   #print "_"*120 
   #print set2               # Dhankar Code 
   #print type(set2)         # Dhankar Code 

   return (set1,set2)



#### As seen below this Function - divideset(my_data,2,'yes') . Which takes THREE INPUT PARAMETERS - or ARGUMENTS. 
 - 1st Parameter - ROWS , the Data Sets Rows , basically the RAW DataSet of Type LIST. 
 - 2nd Parameter - COLUMN , numeric value for Column of the Data Set- on which the DECISION to SPLIT is based 
 - 3rd Parameter - VALUE , the value to be compared with - in this case can be Numeric (int or float) or Non Numeric , in case of Non Numeric values in the Column - a Boolean Equality check is done. 



In [8]:
# Using Function -def divideset(rows,column,value):

#divideset(my_data,0,40) # Col - AGE , Value - 40 Years 
#divideset(my_data,1,40) # Col - BusinessTravel , Value - 1/2/3 


In [9]:
# Counts the UNIQUE VALUES for each PRED CLASS or TARGET VAR Values in LAST COLUMN of DATA SET 
# if a Certain Data Set doesnt have TARGET VAR as LAST FEATURE or COLUMN we need to make it so .....

def uniquecounts(rows):
   results={}
   for row in rows:
      # The result is the last column
      rr=row[len(row)-1]
      
      if rr not in results: results[rr]=0
      results[rr]+=1
   return results

In [10]:
# Unique Count of Classes of Target Feature 

uniquecounts(my_data)


{0: 2466, 1: 474}

### Gini Impurity 

Its the expected Error rate for a SET of Obs . Error that may occur in predicting a class for an Obs. 

- If say Total - 12 Obs , of which all 12 belong to 1 Class Only then Gini-Impurity =0
- If say Total - 12 Obs , of which 3 belong to One Class each , total 4 Classes , then Gini-Impurity =0.75
- If say Total - 12 Obs , of which 6 belong to One Class each , total 4 Classes , then Gini-Impurity =0.50 


In [11]:
# Probability that a randomly placed item will be in the wrong category
# Wiki --- Used by the CART (classification and regression tree) algorithm, Gini impurity 
# is a ---- "measure of how often"------ a randomly chosen element from the set would be incorrectly labeled 
# if it was randomly labeled according to the distribution of labels in the subset.

def giniimpurity(rows):
  total=len(rows)
  print total          #______________________________________Test
  counts=uniquecounts(rows)
  print counts         #______________________________________Test
  imp=0
  for k1 in counts:
    p1=float(counts[k1])/total
    print "_"*100+"_||_" #______________________________________Test
    print (counts[k1])   #______________________________________Test
      
    for k2 in counts:
      if k1==k2: continue
      p2=float(counts[k2])/total
      print "_"*100      #______________________________________Test
      print (counts[k2]) #______________________________________Test
      imp+=p1*p2
  print "_"*60           #______________________________________Test
  print (counts[k1])     #______________________________________Test 
  print "_"*60           #______________________________________Test
  print (counts[k2])     #______________________________________Test
  return imp


In [12]:
giniimpurity(my_data)

2940
{0: 2466, 1: 474}
_____________________________________________________________________________________________________||_
2466
____________________________________________________________________________________________________
474
_____________________________________________________________________________________________________||_
474
____________________________________________________________________________________________________
2466
____________________________________________________________
474
____________________________________________________________
474


0.27046230737192833

In [13]:
# Original entropy Func.
#
#

def entropy(rows):
   from math import log             ## Import within Func.Entropy - as Function can be called anywhere in PipeLine
   log2=lambda x:log(x)/log(2)      ## TBD --------------- Comment Pending .........
   results=uniquecounts(rows)       ## Same as -"counts=uniquecounts(rows)" -in GiniImpurity Function
   # Now calculate the entropy
   ent=0.0
   for r in results.keys():         ## This DATA_SET has TWO results.keys() as only 2 CLASSES of TARGET VAR. 
      p=float(results[r])/len(rows) ## Probability of Occurence of TARGET CLASS 0 OR 1 
      ent=ent-p*log2(p)
   return ent
   print "Final ent == ",ent

In [14]:
entropy(my_data)

0.6372308920655037

In [15]:
#SandBox for --- def entropy(rows):

results=uniquecounts(my_data)   # Same as -"counts=uniquecounts(rows)" -in GiniImpurity Function
for r in results.keys():
        print "TEST - as many times this prints - that many results.keys() and Classes "
   

TEST - as many times this prints - that many results.keys() and Classes 
TEST - as many times this prints - that many results.keys() and Classes 


In [16]:
#SandBox for --- def entropy(rows):

from math import log             ## Import within Func.Entropy - as Function can be called anywhere in PipeLine
log2=lambda x:log(x)/log(2)      ## x = Parameter is passed Value = p below.

results=uniquecounts(my_data)  # Same as -"counts=uniquecounts(rows)" -in GiniImpurity Function
ent=0.0
for r in results.keys():    
    p=float(results[r])/len(my_data) # p=Probability of Occurence of TARGET CLASS 0 OR 1
    ent=ent-p*log2(p)
    print "p = ",p        # First time through FOR LOOP - Prints the p for results.keys - 1st VALUE then the next . 
    print "ent =",ent # Seen above also- get FINAL_VALUE of ENTROPY after 2 Iterations of FOR == ent = 0.637

p =  0.838775510204
ent = 0.212749834769
p =  0.161224489796
ent = 0.637230892066


#### Entropy is the sum of p(x)log(p(x)) across all the different possible results - all possible Unique Values for TARGET VAR like 0 and 1 - Live or Exited.    In lay man terms - Entropy is Inversely Proportional to Purity or HomoGenity of the SET.  

##### If All Observations in the SET are of 1 CLASS only - say PREMIUM - then the SET is Totally PURE or HOMOGENOUS .  In such a case ENTROPY is Lowest 


In [17]:
## SAND BOX - 

# Split -1 
set1,set2=divideset(my_data,0,20) # Column = 0 = AGE , Value >= 20 Assigned to SET -1 

print "_"*100+"__|||__"
print "# Split -1"
print "Metrics_SET(1):"
print ('Gini_Impurity :-{:.4f}'.format(giniimpurity(set1)))
print ("Entropy       :-{:.4f}".format(entropy(set1)))
print "UniqueCount_TargetVar:-",uniquecounts(set1)
print "_"*50
print "Metrics_SET(2):"
print ('Gini_Impurity :-{:.4f}'.format(giniimpurity(set2)))
print ("Entropy       :-{:.4f}".format(entropy(set2)))
print "UniqueCount_TargetVar:-",uniquecounts(set2)


# Split -2
set1,set2=divideset(my_data,1,2) # Column = 1 = BussinesTravel , Value >= 2 Assigned to SET -1 

print "_"*100+"__|||__"
print "# Split -2"
print "Metrics_SET(1):"
print ('Gini_Impurity :-{:.4f}'.format(giniimpurity(set1)))
print ("Entropy       :-{:.4f}".format(entropy(set1)))
print "UniqueCount_TargetVar:-",uniquecounts(set1)
print "_"*50
print "Metrics_SET(2):"
print ('Gini_Impurity :-{:.4f}'.format(giniimpurity(set2)))
print ("Entropy       :-{:.4f}".format(entropy(set2)))
print "UniqueCount_TargetVar:-",uniquecounts(set2)


# Split -2
set1,set2=divideset(my_data,2,1000) # Column = 1 = DailyRate, Value >= 1000 Assigned to SET -1 

print "_"*100+"__|||__"
print "# Split -3"
print "Metrics_SET(1):"
print ('Gini_Impurity :-{:.4f}'.format(giniimpurity(set1)))
print ("Entropy       :-{:.4f}".format(entropy(set1)))
print "UniqueCount_TargetVar:-",uniquecounts(set1)
print "_"*50
print "Metrics_SET(2):"
print ('Gini_Impurity :-{:.4f}'.format(giniimpurity(set2)))
print ("Entropy       :-{:.4f}".format(entropy(set2)))
print "UniqueCount_TargetVar:-",uniquecounts(set2)



______________________________________________________________________________________________________|||__
# Split -1
Metrics_SET(1):
2906
{0: 2452, 1: 454}
_____________________________________________________________________________________________________||_
2452
____________________________________________________________________________________________________
454
_____________________________________________________________________________________________________||_
454
____________________________________________________________________________________________________
2452
____________________________________________________________
454
____________________________________________________________
454
Gini_Impurity :-0.2636
Entropy       :-0.6252
UniqueCount_TargetVar:- {0: 2452, 1: 454}
__________________________________________________
Metrics_SET(2):
34
{0: 14, 1: 20}
_____________________________________________________________________________________________________||_
14
_

In [18]:
def buildtree(rows,scoref=entropy):
  if len(rows)==0: return decisionnode()
  current_score=scoref(rows)

  # Set up some variables to track the best criteria
  best_gain=0.0
  best_criteria=None
  best_sets=None
  
  column_count=len(rows[0])-1       ### Commented below in Markdown Cell 
  for col in range(0,column_count): ###
    # Generate the list of different values in this column
    column_values={}                ###   
    for row in rows:                ###   
       column_values[row[col]]=1
    # Now try dividing the rows up for each value in this column
    for value in column_values.keys():
      (set1,set2)=divideset(rows,col,value)
      #print  "Col_Value_Keys :---",column_values.keys()  ##### OWN CODE for Print ............
      
      # Information gain
      p=float(len(set1))/len(rows)
      gain=current_score-p*scoref(set1)-(1-p)*scoref(set2)
      if gain>best_gain and len(set1)>0 and len(set2)>0:
        best_gain=gain
        best_criteria=(col,value)
        best_sets=(set1,set2)
  # Create the sub branches   
  if best_gain>0:
    trueBranch=buildtree(best_sets[0]) # Right Branch 
    falseBranch=buildtree(best_sets[1]) # Left Branch 
    return decisionnode(col=best_criteria[0],value=best_criteria[1],
                        tb=trueBranch,fb=falseBranch)
  else:
    return decisionnode(results=uniquecounts(rows))

 To determine which attribute is the best to divide on, the information gain is calculated. Information gain
 is the difference between the current entropy and the weighted-average entropy of  the two new groups.

In [19]:
# SandBox - Info_Gain
#
scoref=entropy
rows=my_data
p=float(len(set1))/len(rows)                        ### As-Is from above 
current_score=scoref(rows)                          ### As-Is from above 

# gain=current_score-p*scoref(set1)-(1-p)*scoref(set2)
# gain = is difference between current entropy and weighted-average entropy of the two sets. 

print "Length of set1:--------------" ,float(len(set1))
print "Length of rows or my_data:--- ",len(my_data)
print "Current Score :--------------", scoref(my_data) # 0.637230892066
print "Value of Gain :--------------" , current_score-p*scoref(set1)-(1-p)*scoref(set2) # 0.00195 for test-set1 n 2

best_sets=(set1,set2)
print "_"*90
print "Best Sets is a :--",type(best_sets)
print "_"*90
#print (best_sets[0]) # OK Dont Print Large Output 
print "_"*90
#print (best_sets[1]) # OK Dont Print Large Output 

#buildtree(best_sets[0])

Length of set1:-------------- 1046.0
Length of rows or my_data:---  2940
Current Score :-------------- 0.637230892066
Value of Gain :-------------- 0.00195282504731
__________________________________________________________________________________________
Best Sets is a :-- <type 'tuple'>
__________________________________________________________________________________________
__________________________________________________________________________________________


### column_count=len(rows[0])-1

Count COLUMNS or Number of Values in first_ROW [0] of my_data ,minus 1(Target_var) 

### for col in range(0,column_count):

for loop - iterate within range 0 to column_count . Loop will start in Row 0 Column 0 or Value [0,0]


### column_values={}

Create Empty LIST to hold values 

### for row in rows: 
###         column_values[row[col]]=1

Next nested For Loop will RUN or Iterate through ALL ROWS or my_data 
While Iterating through All ROWS - it shall provide to the FUNCTION - divideset(rows,col,value) 
The required THREE PARAMETERS 

- rows == As "for row in rows" will iterate through all rows 
- col  == As "for col in range(0,column_count):" will iterate through all columns or values of each row
- value == As "for value in column_values.keys():" will iterate over all values in each column 

We PRINT out the -- for value in column_values.keys(): in the Code Cell below and see the PRINT Output change from 
- Col Val Keys :--- ['(direct)', 'digg', 'google', 'slashdot', 'kiwitobes']
#### to 
- Col Val Keys :--- ['New Zealand', 'UK', 'USA', 'France']
#### after FIVE Iterations 
    



In [20]:
#tree_1=buildtree(my_data) # Dont print - large OutPut 

In [21]:
#
def printtree(tree,indent=''):
   # Is this a leaf node?
   if tree.results!=None:
      print str(tree.results)
   else:
      # Print the criteria
      print str(tree.col)+':'+str(tree.value)+'? '

      # Print the branches
      print indent+'T->',
      printtree(tree.tb,indent+'  ')
      print indent+'F->',
      printtree(tree.fb,indent+'  ')


In [22]:
# Function - printtree()

tree_1=buildtree(my_data) # Dont print - large OutPut 
printtree(tree_1)

18:1? 
T-> 15:3760? 
  T-> 12:8? 
    T-> 14:2? 
      T-> 4:13? 
        T-> 19:15? 
          T-> 17:1? 
            T-> {0: 10}
            F-> {1: 2}
          F-> 16:6110? 
            T-> {1: 12}
            F-> {0: 4}
        F-> 23:12? 
          T-> 21:2? 
            T-> 0:32? 
              T-> {0: 30}
              F-> {1: 2}
            F-> {1: 6}
          F-> {0: 58}
      F-> 28:2? 
        T-> {1: 22}
        F-> 4:24? 
          T-> {1: 6}
          F-> 4:2? 
            T-> 0:36? 
              T-> {0: 16}
              F-> 4:5? 
                T-> {1: 8}
                F-> {0: 8}
            F-> {1: 4}
    F-> 17:7? 
      T-> 7:3? 
        T-> 0:58? 
          T-> {1: 4}
          F-> {0: 26}
        F-> 9:79? 
          T-> 0:48? 
            T-> {1: 2}
            F-> {0: 4}
          F-> {1: 10}
      F-> 0:24? 
        T-> 10:2? 
          T-> 2:728? 
            T-> {0: 142}
            F-> 2:725? 
              T-> {1: 2}
              F-> 4:14? 
          

In [23]:
def getwidth(tree):
  if tree.tb==None and tree.fb==None: return 1 # 1 is Default WIDTH of TREE if NO BRANCHES 
  return getwidth(tree.tb)+getwidth(tree.fb)

def getdepth(tree):
  if tree.tb==None and tree.fb==None: return 0
  return max(getdepth(tree.tb),getdepth(tree.fb))+1


from PIL import Image,ImageDraw

def drawtree(tree,jpeg='tree.jpg'):
  w=getwidth(tree)*100
  h=getdepth(tree)*100+120

  img=Image.new('RGB',(w,h),(255,255,255))
  draw=ImageDraw.Draw(img)

  drawnode(draw,tree,w/2,20)
  img.save(jpeg,'JPEG')
  
def drawnode(draw,tree,x,y):
  if tree.results==None:
    # Get the width of each branch
    w1=getwidth(tree.fb)*100
    w2=getwidth(tree.tb)*100

    # Determine the total space required by this node
    left=x-(w1+w2)/2
    right=x+(w1+w2)/2

    # Draw the condition string
    draw.text((x-20,y-10),str(tree.col)+':'+str(tree.value),(0,0,0))

    # Draw links to the branches
    draw.line((x,y,left+w1/2,y+100),fill=(255,0,0))
    draw.line((x,y,right-w2/2,y+100),fill=(255,0,0))
    
    # Draw the branch nodes
    drawnode(draw,tree.fb,left+w1/2,y+100)
    drawnode(draw,tree.tb,right-w2/2,y+100)
  else:
    txt=' \n'.join(['%s:%d'%v for v in tree.results.items()])
    draw.text((x-20,y),txt,(0,0,0))



In [33]:
drawtree(tree_1,jpeg='tree_HR.jpg')

In [24]:
def classify(observation,tree): # OBS = 1 Row of Data all Features besides Target. # Tree = tree created above. 
  if tree.results!=None:
    return tree.results
  else:
    v=observation[tree.col]
    branch=None
    if isinstance(v,int) or isinstance(v,float):
      if v>=tree.value: branch=tree.tb
      else: branch=tree.fb
    else:
      if v==tree.value: branch=tree.tb
      else: branch=tree.fb
    return classify(observation,branch)


In [32]:
obs0 = my_data[0,:]
obs1 = my_data[1,:]
obs2= my_data[2,:]
obs3 = my_data[3,:]
obs4 = my_data[4,:]

#print obs1

print classify(obs0,tree_1)
print classify(obs1,tree_1) # 
print classify(obs2,tree_1) # 
print classify(obs3,tree_1) # 
print classify(obs4,tree_1) # 
# As seen below 5 Correct PREDS 
# Further TBD ...................................Prune etc ... 
# Compare results with INTERNAL to sciKitLearn TREE Functions 

{1: 4}
{0: 160}
{1: 8}
{0: 22}
{0: 40}


In [ ]:

def prune(tree,mingain):
  # If the branches aren't leaves, then prune them
  if tree.tb.results==None:
    prune(tree.tb,mingain)
  if tree.fb.results==None:
    prune(tree.fb,mingain)
    
  # If both the subbranches are now leaves, see if they
  # should merged
  if tree.tb.results!=None and tree.fb.results!=None:
    # Build a combined dataset
    tb,fb=[],[]
    for v,c in tree.tb.results.items():
      tb+=[[v]]*c
    for v,c in tree.fb.results.items():
      fb+=[[v]]*c
    
    # Test the reduction in entropy
    delta=entropy(tb+fb)-(entropy(tb)+entropy(fb)/2)

    if delta<mingain:
      # Merge the branches
      tree.tb,tree.fb=None,None
      tree.results=uniquecounts(tb+fb)

In [14]:
def mdclassify(observation,tree):
  if tree.results!=None:
    return tree.results
  else:
    v=observation[tree.col]
    if v==None:
      tr,fr=mdclassify(observation,tree.tb),mdclassify(observation,tree.fb)
      tcount=sum(tr.values())
      fcount=sum(fr.values())
      tw=float(tcount)/(tcount+fcount)
      fw=float(fcount)/(tcount+fcount)
      result={}
      for k,v in tr.items(): result[k]=v*tw
      for k,v in fr.items(): result[k]=v*fw      
      return result
    else:
      if isinstance(v,int) or isinstance(v,float):
        if v>=tree.value: branch=tree.tb
        else: branch=tree.fb
      else:
        if v==tree.value: branch=tree.tb
        else: branch=tree.fb
      return mdclassify(observation,branch)

def variance(rows):
  if len(rows)==0: return 0
  data=[float(row[len(row)-1]) for row in rows]
  mean=sum(data)/len(data)
  variance=sum([(d-mean)**2 for d in data])/len(data)
  return variance



In [43]:
###SANDBOX IMPORTANT --- Refer URL -- https://docs.python.org/2/tutorial/classes.html


class MyClass:
    """A simple example class - No _init_"""
    # Everything defined below is an ATTRIBUTE in the Classes NAMESPACE
    # They can all be called upon as CLASS_DOT_ATTRIBUTE
    
    i = 12345
    g = "Cheers Mate"
    
    
    def f(self):
        return 'hello world'
    
print MyClass.i
print MyClass.g    

print MyClass.f

12345
Cheers Mate
<unbound method MyClass.f>
